In [15]:
import os
import random
import numpy as np
import pandas as pd
import math
from tqdm import tnrange, tqdm_notebook


from glob import glob
from IPython.display import display

In [21]:
#Load Validation dataset & pCTRs
Path = 'C://Datasets//Multiagent//calc//pCTRs'

#Importing pCTR data.
os.chdir(Path)
Datasets_CTR = {}

for Filename in glob('*validation_pCTR.csv'):
    Datasets_CTR[Filename[:-4]] = pd.read_csv(Filename, sep = ',')

#Importing Validation data.
os.chdir('c://Datasets//Multiagent//rtb')
Datasets_features = {}
for Filename in glob('Validation.csv'):
    Datasets_features[Filename[:-4].title()] = pd.read_csv(Filename, sep = ',')
    
#Defining variables.
Validation = Datasets_features['Validation']

In [22]:
# Define Bid Strategies
def linear_bidding(base_bid, pCTR, Init_CTR):
    return (base_bid*pCTR/Init_CTR)

def nlogn_bidding(base_bid, pCTR, Init_CTR):
    return (base_bid*(pCTR/Init_CTR)*np.log(pCTR/Init_CTR))

def exp_bidding(base_bid, pCTR, Init_CTR):
    return (base_bid*np.e**(pCTR/Init_CTR))


In [23]:
class Agent:
    
    initPCR = 0.0007375623256
    initBudget = 6250
        
    def __init__(self, id, pCTR_strategy, bid_strategy, params):
        self.budget = self.initBudget
        self.id = id
        self.pCTR_strategy = pCTR_strategy
        self.bid_strategy = bid_strategy
        self.params = params        
        self.pCTRs = list(Datasets_CTR[self.pCTR_strategy +'_validation_pCTR']['pCTR'].values)
        self.bids = tuple()
        self.initBids()
            
    def initBids(self):
        bids = []
        for pCTR in self.pCTRs:
            if(self.bid_strategy == "Linear"):
                bids.append(linear_bidding(self.params[0],pCTR,self.initPCR))
            elif(self.bid_strategy == "NLogN"):
                bids.append(nlogn_bidding(self.params[0],pCTR,self.initPCR)) 
            elif(self.bid_strategy == "Exp"):
                bids.append(exp_bidding(self.params[0],pCTR,self.initPCR))
            else:
                raise("Bid strategy not found for agent ", self.id)
        self.bids = bids    
        return
        
     
    # Update budget based on bid, returns True if budget allows
    def updateBudget(self,bid,overbid=False):
        if(not overbid):
            if (self.budget - bid) < 0:
                print("ERROR: not enough budget for bid")
                return False
            else:
                self.budget = self.budget - bid
                return True
        else:
            self.budget = self.budget - bid
            return

        
    def canBid(self,bid):
        if(self.budget>bid):
            return True
        return False
    
    #Ensure that bid is within budget
    def getMaxBid(self,idx,overbid=False):
        if (overbid and self.budget>0) or (self.budget > self.bids[idx]):
            return self.bids[idx]
        else:
            return self.budget
    
    def addBids(self,bids):
        self.bids = bids
        return
    

In [28]:
class AgentsClass:    
    
    def __init__(self):
        self.Agents = []
        return
    
    def addAgent(self,agent):
        self.Agents.append(agent)
    
    def get(self):
        return self.Agents
    

In [40]:

agentsList = AgentsClass()


agentsList.addAgent(Agent(1,  "LR",  "Linear", [1]))
agentsList.addAgent(Agent(2,  "LR",  "Linear", [100]))
agentsList.addAgent(Agent(3,  "LR",  "Exp",  [30]))
agentsList.addAgent(Agent(4,  "LR",  "Linear", [105]))
agentsList.addAgent(Agent(5,  "LR",  "Exp",   [29]))
agentsList.addAgent(Agent(6,  "LR",  "Linear",  [121]))
agentsList.addAgent(Agent(7,  "LR",  "NLogN",  [125]))
agentsList.addAgent(Agent(8,  "LR",  "Linear",  [95]))
agentsList.addAgent(Agent(9,  "LR",  "Exp",   [31]))
agentsList.addAgent(Agent(10, "LR",  "Linear",  [110]))
agentsList.addAgent(Agent(11, "LR",  "Linear",  [105]))
agentsList.addAgent(Agent(12, "LGBM", "Linear", [120]))
agentsList.addAgent(Agent(13, "LGBM", "Linear", [115]))
agentsList.addAgent(Agent(14, "LGBM", "Exp",  [35]))
agentsList.addAgent(Agent(15, "LGBM", "Exp",  [30]))
agentsList.addAgent(Agent(16, "LGBM", "NLogN", [10]))
agentsList.addAgent(Agent(17, "LGBM", "NLogN", [20]))
agentsList.addAgent(Agent(18, "NN",  "NLogN", [7500]))
agentsList.addAgent(Agent(19, "NN",  "NLogN", [9500]))
agentsList.addAgent(Agent(20, "NN",  "Exp",  [30]))
agentsList.addAgent(Agent(21, "NN",  "Exp",  [35]))
agentsList.addAgent(Agent(22, "NN",  "Linear", [140]))
agentsList.addAgent(Agent(23, "NN",  "Linear", [150]))


#Agents = [Agent1,Agent2,Agent3,Agent4,Agent5,Agent6,Agent7,Agent8,Agent9,Agent10,
      #    Agent11, Agent12, Agent13,Agent14,Agent15,Agent16,Agent17,Agent18,Agent19]

debug = True


#df = pd.DataFrame(AgentsBids, columns=["Agent","Bid"])

#AgentsBidsDf = pd.DataFrame(AgentsBids, columns=["Agent","Bid"])



def getBidsCurrentAuction(agents, idx, overbid = False, verbose=False):
    bidsCurrentAuction = []
    for agent in agents:        
        if(verbose):
            print('Bid[',agent.id,']:', agent.getMaxBid(idx,overbid))
        bidsCurrentAuction.append([agent, agent.getMaxBid(idx,overbid)])
    return pd.DataFrame(bidsCurrentAuction, columns=["Agent","Bid"])

def secondPriceWinner(AgentsBidsDf, floorPrice, idx=0, overbid=False,verbose=False):
    #if(idx> 288398 ):
    #    verbose=True
    bestBids= AgentsBidsDf.sort_values(by="Bid", ascending=False)
    if(verbose):
        print(bestBids)
    highestBidAgent = bestBids.iloc[0,0]
    curr_bid = bestBids.iloc[0,1]
    secondPriceOthers = bestBids.iloc[1,1]
    #print(curr_bid, floorPrice)
    if(curr_bid >= floorPrice):
        actualPayPrice = max(floorPrice,secondPriceOthers)
        highestBidAgent.updateBudget(actualPayPrice/1000, overbid)
        if (verbose):
            print(idx,"\tWon: ", highestBidAgent.id, " Paid:" , '{:.3f}'.format(floorPrice), " Upd budget:", '{:.3f}'.format(highestBidAgent.budget))
        return highestBidAgent
    else:        
        if(verbose):
            print(idx, "\tNo winner, Floor:",floorPrice)
        return None
    
    

def individualAuctions(verbose=False):
    overbid = False
    results_1 = {}
    for idx in tnrange(len(Validation), desc='Simulating auction'):
    #for idx in tnrange(202, desc='Simulating auction'):    
        #curr_bid = Validation.iloc[idx]
        #if(verbose):
        #    print (Validation.iloc[idx])
        df = getBidsCurrentAuction(agentsList.get(),idx,overbid,verbose=False)
        floorPrice = Validation['payprice'][idx]
        winner = secondPriceWinner(df,floorPrice,idx,overbid,verbose=verbose)
        if (winner != None):
            results_1[idx]=(idx,winner.id,winner.budget)            
        else:
            results_1[idx]=(idx,np.NaN,np.NaN)
        
    results_df=pd.DataFrame.from_dict(results_1).transpose()
        
    return results_df
        


C:\Users\mau_c\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: RuntimeWarning: overflow encountered in double_scalars
  if __name__ == '__main__':


In [41]:
df = individualAuctions(False)
df.rename(columns={0:'Ad', 1:'Winner', 2:'Budget'},inplace=True)
df.head()

,Ad,Winner,Budget
0,0.0,23.0,6249.884755
1,1.0,NaN,NaN
2,2.0,NaN,NaN
3,3.0,6.0,6249.952432
4,4.0,9.0,6249.959755


In [7]:
#df.rename(columns={0:'Ad', 1:'Winner', 2:'Budget'}, inplace=True)
#df.head()

In [42]:
results ={}
for idx in range(len(agentsList.get())):    
    agent = agentsList.get()[idx]    
    a_idx_won = df[df["Winner"] == agent.id]['Ad']    
    a_won = Validation.iloc[a_idx_won]
    a_clicks = a_won[a_won['click'] == 1]['click'].sum()
    a_imps = len(a_won)
    a_spent = -1 * (agent.budget-Agent.initBudget)    
    results[idx] = ('Agent'+str(agent.id),agent.pCTR_strategy,agent.bid_strategy,agent.params,a_clicks,a_imps,round(a_spent,3))
#results
results_df = pd.DataFrame.from_dict(results).transpose()
results_df.rename(columns={0:'Agent', 1:'pCTR', 2:'Bidding',3:'BB', 4:'Clicks', 5:'Imps' , 6:'Spent' }, inplace=True)

results_df.sort_values('Clicks', ascending=False)

,Agent,pCTR,Bidding,BB,Clicks,Imps,Spent
21,Agent22,NN,Linear,[140],23,307,991.463
11,Agent12,LGBM,Linear,[120],20,57370,6022.93
6,Agent7,LR,NLogN,[125],20,3017,5292.39
22,Agent23,NN,Linear,[150],18,30718,5241.03
14,Agent15,LGBM,Exp,[30],15,2578,5664.26
19,Agent20,NN,Exp,[30],12,2343,5644.12
13,Agent14,LGBM,Exp,[35],11,33804,5934.29
5,Agent6,LR,Linear,[121],11,46025,5326.23
4,Agent5,LR,Exp,[29],9,2152,5523.28
10,Agent11,LR,Linear,[105],6,39,239.099


In [43]:
results_df['Clicks'].sum()


178

In [ ]:
#ind = ()
#print(type(ind))
#print(ind[0])
#Validation.iloc[df[df.iloc[:,1] == 1][0]].count()


In [ ]:
ind_temp = 1688
df.iloc[ind_temp:ind_temp+60]

## Error in 288402

In [ ]:
bestBids= df.sort_values(by="Bid", ascending=False)
bestBids

In [ ]:
print("best bid by: ", bestBids.iloc[0,0].id, " second " , bestBids.iloc[1,0].id)
print("Win price: ", bestBids.iloc[0,1], " Price to pay " , bestBids.iloc[1,1])
